In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
import numpy as np
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from azure.cognitiveservices.vision.face.models import FaceAttributeType

In [2]:
#import vision API 
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array

In [3]:
#Retrive from Face Resource - Resource Management
faceKEY = "1ab3848ea22649299da961e8feb813a4"
faceENDPOINT = "https://face-rec-app.cognitiveservices.azure.com/"

In [4]:
# Create an authenticated FaceClient.
face_client = FaceClient(faceENDPOINT, CognitiveServicesCredentials(faceKEY))

In [5]:
#Retrive key and endpoint from Computer Vision Resource
compvision_subscription_key = "834987a3f5ef426e866617fd0739af1e"
compvision_endpoint = "https://visualpattern.cognitiveservices.azure.com/"

In [6]:
#Create an authenticated Computer Vision Client
computervision_client = ComputerVisionClient(compvision_endpoint, CognitiveServicesCredentials(compvision_subscription_key))

In [7]:
# Attributes you want returned with the API call, a list of FaceAttributeType enum (string format)
face_attributes = ['headPose', 'glasses', 'occlusion', 'blur', 'exposure', 'noise']

In [18]:
# Convert width height to a point in a rectangle
def faceApp(image):
    local_image = open(image, "rb")
    image_size = Image.open(image).size
    image_width, image_height = image_size
    # Detect a face with attributes, returns a list[DetectedFace]
    detected_faces = face_client.face.detect_with_stream(local_image, return_face_attributes=face_attributes)
    if not detected_faces:
        raise Exception(
                'No face detected from image {}'.format(os.path.basename(image)))
    listofIssues = []
    
    #possible issues
    
    horCentre = 'Face is not horizontally centred'
    verCentre = 'Face is not vertically centred'
    horverCentred = 'Face is horizontally and vertically centred'
    horverNotCentred = 'Face is neither horizontally or vertically centred'
    
    rolled = 'head is tilted to the side'
    yawed = 'face is turned to the side'
    pitched = 'head is foreward or backward titled'
    tiltCentred = 'head and face is correctly positioned'
    rolledYawed = 'head is tilted to the side and face is turned to the side'
    rolledPitched = 'head is tilted to the side and head is foreward or backward tilted'
    pitchedYawed = 'head is foreward or backward tilted and face is turned to the side'
    rolledYawedPitched = 'head is tilted to the side, face is turned to the side, and head is forward or backward tilted'
    
    glasses = 'Face is obstructed by glasses'
    noGlasses = 'Face does not have glasses'
    
    eyesCovered = 'Eyes are occluded'
    eyesVisible = 'Eyes are not occluded'
    
    foreheadCovered = 'Forehead is occluded'
    foreheadVisible = 'Forehead is not occluded'
    
    for face in detected_faces:
        rect = face.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height
        rightspace = image_width - right
        bottomspace = image_height - bottom 
           
        if (abs(left - rightspace) > 50) and (abs(top - bottomspace) >75):
            listofIssues.append(horverNotCentred)
        elif abs(left - rightspace) > 50: 
            listofIssues.append(horCentre)
        elif abs(top - bottomspace) > 75:
            listofIssues.append(verCentre)
        else:
            listofIssues.append(horverCentred)
    
    head_pose_string = str(face.face_attributes.head_pose)
    a, r, y, p = head_pose_string.split(',')
    roll = r[8:]
    roll_num = abs(float(roll))
    yaw = y[7:]
    yaw_num = abs(float(yaw))
    pitch = p[9:-1]
    pitch_num = abs(float(pitch))
    
    if (roll_num > 5) and (yaw_num > 5) and (pitch_num > 5):
        listofIssues.append(rolledYawedPitched)
    elif (roll_num > 5) and (yaw_num > 5):
        listofIssues.append(rolledYawed)
    elif (roll_num > 5) and (pitch_num > 5):
        listofIssues.append(rolledPitched)
    elif (yaw_num > 5) and (pitch_num > 5):
        listofIssues.append(pitchedYawed)
    elif roll_num > 5:
        listofIssues.append(rolled)
    elif yaw_num > 5:
        listofIssues.append(yawed)
    elif pitch_num > 5:
        listofIssues.append(pitched)
    else:
        listofIssues.append(tiltCentred)
        
    glasses_string = str(face.face_attributes.glasses)
    if glasses_string.__contains__("no_glasses") == True:
        listofIssues.append(noGlasses)
    else:
        listofIssues.append(glasses)
        
    if face.face_attributes.occlusion.eye_occluded == True:
        listofIssues.append(eyesCovered)
    else:
        listofIssues.append(eyesVisible)
        
    if face.face_attributes.occlusion.forehead_occluded == True:
        listofIssues.append(foreheadCovered)
    else:
        listofIssues.append(foreheadVisible)
    
    print(listofIssues)

In [17]:
import glob

for image_file in glob.iglob('C:\\Users\\nidhimh\\Documents\\Face\\faces\\*.jpg'):
    faceApp(image_file)

['Face is not horizontally centred', 'head and face is correctly positioned', 'Face does not have glasses', 'Eyes are not occluded', 'Forehead is not occluded']
['Face is not horizontally centred', 'face is turned to the side', 'Face does not have glasses', 'Eyes are not occluded', 'Forehead is not occluded']
['Face is not horizontally centred', 'head and face is correctly positioned', 'Face does not have glasses', 'Eyes are not occluded', 'Forehead is not occluded']
['Face is not horizontally centred', 'head and face is correctly positioned', 'Face does not have glasses', 'Eyes are not occluded', 'Forehead is not occluded']
['Face is not horizontally centred', 'head and face is correctly positioned', 'Face does not have glasses', 'Eyes are not occluded', 'Forehead is not occluded']
['Face is not horizontally centred', 'head is foreward or backward titled', 'Face does not have glasses', 'Eyes are not occluded', 'Forehead is not occluded']
['Face is not vertically centred', 'head is for

APIErrorException: (429) Requests to the Face - Detect Operation under Face API - v1.0 have exceeded rate limit of your current Face F0 pricing tier. Please retry after 46 seconds. To increase your rate limit switch to a paid tier.